<a href="https://colab.research.google.com/github/ArkanDash/Advanced-RVC-Inference/blob/master/Advanced-RVC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced RVC Inference:

<big> for quicker and effortless model downloads

In [ ]:
#@title Check GPU
!nvidia-smi

In [ ]:
#@title Install Dependencies
from IPython.display import clear_output
from ipywidgets import Button




# ===== Clone Repository ===== #

!git clone -b master https://github.com/ArkanDash/Advanced-RVC-Inference

%cd  /content/Advanced-RVC-Inference



# ===== Install requirements ===== #
!pip install pip==24.0
!pip install -r ./requirements.txt

In [ ]:
#@title Download Hubert & base Model




!python ./models.py

In [ ]:
#@title audio separator for infernece



import os
import glob
import yt_dlp

def downloader(url):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
        'outtmpl': os.path.join(f'{path}/temp', '%(title)s.%(ext)s'),
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

def checker(url):
    return "http" in url

def uvr_cli(audio_input, output_folder, model, output_format, segment_size, overlap, batch_size, override_model_segment_size, use_autocast, extensions):
    found_files = []

    dictmodel = {
        'BS-Roformer-Viperx-1297': 'model_bs_roformer_ep_317_sdr_12.9755.ckpt',
        'BS-Roformer-Viperx-1296': 'model_bs_roformer_ep_368_sdr_12.9628.ckpt',
        'BS-Roformer-Viperx-1053': 'model_bs_roformer_ep_937_sdr_10.5309.ckpt',
        'Mel-Roformer-Viperx-1143': 'model_mel_band_roformer_ep_3005_sdr_11.4360.ckpt',
        'BS-Roformer-De-Reverb': 'deverb_bs_roformer_8_384dim_10depth.ckpt',
        'Mel-Roformer-Crowd-Aufr33-Viperx': 'mel_band_roformer_crowd_aufr33_viperx_sdr_8.7144.ckpt',
        'Mel-Roformer-Denoise-Aufr33': 'denoise_mel_band_roformer_aufr33_sdr_27.9959.ckpt',
        'Mel-Roformer-Denoise-Aufr33-Aggr' : 'denoise_mel_band_roformer_aufr33_aggr_sdr_27.9768.ckpt',
        'Mel-Roformer-Karaoke-Aufr33-Viperx': 'mel_band_roformer_karaoke_aufr33_viperx_sdr_10.1956.ckpt',
        'MelBand Roformer | Vocals by Kimberley Jensen' : 'vocals_mel_band_roformer.ckpt',
        'MelBand Roformer Kim | FT by unwa' : 'mel_band_roformer_kim_ft_unwa.ckpt',
        'MelBand Roformer Kim | Inst V1 by Unwa' : 'melband_roformer_inst_v1.ckpt',
        'MelBand Roformer Kim | Inst V1 (E) by Unwa' : 'melband_roformer_inst_v1e.ckpt',
        'MelBand Roformer Kim | Inst V2 by Unwa' : 'melband_roformer_inst_v2.ckpt',
        'MelBand Roformer Kim | InstVoc Duality V1 by Unwa' : 'melband_roformer_instvoc_duality_v1.ckpt',
        'MelBand Roformer Kim | InstVoc Duality V2 by Unwa' : 'melband_roformer_instvox_duality_v2.ckpt',
        'MelBand Roformer | De-Reverb by anvuew' : 'dereverb_mel_band_roformer_anvuew_sdr_19.1729.ckpt',
        'MelBand Roformer | De-Reverb Less Aggressive by anvuew' : 'dereverb_mel_band_roformer_less_aggressive_anvuew_sdr_18.8050.ckpt',
        'MelBand Roformer | De-Reverb-Echo by Sucial' : 'dereverb-echo_mel_band_roformer_sdr_10.0169.ckpt',
        'MelBand Roformer | De-Reverb-Echo V2 by Sucial' : 'dereverb-echo_mel_band_roformer_sdr_13.4843_v2.ckpt',
        'MelBand Roformer Kim | SYHFT by SYH99999' : 'MelBandRoformerSYHFT.ckpt',
        'MelBand Roformer Kim | SYHFT V2 by SYH99999' : 'MelBandRoformerSYHFTV2.ckpt',
        'MelBand Roformer Kim | SYHFT V2.5 by SYH99999' : 'MelBandRoformerSYHFTV2.5.ckpt',
        'MelBand Roformer Kim | SYHFT V3 by SYH99999' : 'MelBandRoformerSYHFTV3Epsilon.ckpt',
        'MelBand Roformer Kim | Big SYHFT V1 by SYH99999' : 'MelBandRoformerBigSYHFTV1.ckpt',
        'MelBand Roformer Kim | Big Beta 4 FT by unwa' : 'melband_roformer_big_beta4.ckpt',
        'MelBand Roformer Kim | Big Beta 5e FT by unwa' : 'melband_roformer_big_beta5e.ckpt',
        'BS Roformer | Chorus Male-Female by Sucial' : 'model_chorus_bs_roformer_ep_267_sdr_24.1275.ckpt',
        'MelBand Roformer | Aspiration by Sucial' : 'aspiration_mel_band_roformer_sdr_18.9845.ckpt',
        'MelBand Roformer | Aspiration Less Aggressive by Sucial' : 'aspiration_mel_band_roformer_less_aggr_sdr_18.1201.ckpt',
        'MelBand Roformer | Bleed Suppressor V1 by unwa-97chris' : 'mel_band_roformer_bleed_suppressor_v1.ckpt'
    }
    roformer_model = dictmodel[model]

    if checker(audio_input):
        downloader(audio_input)
        audio_input = f"{path}/temp"

    for audio_files in os.listdir(audio_input):
        if audio_files.endswith(extensions):
            found_files.append(audio_files)

    total_files = len(found_files)

    if total_files == 0:
        print("No valid audio files found.")
    else:
        print(f"{total_files} audio files found")

        found_files.sort()

        for audio_files in found_files:
            file_path = os.path.join(audio_input, audio_files)
            prompt = f'audio-separator "{file_path}" --model_filename {roformer_model} --output_dir={output_folder} --output_format={output_format} --mdxc_segment_size={segment_size} --mdxc_overlap={overlap} --mdxc_batch_size={batch_size} --model_file_dir=./models'
            if override_model_segment_size:
                prompt += " --mdxc_override_model_segment_size"
            if use_autocast:
                prompt += " --use_autocast"
            !$prompt

    if audio_input == f"{path}/temp":
        temp_files = glob.glob(f"{path}/temp/*")
        for file in temp_files:
            os.remove(file)

#@markdown Input path for audio files or link:
audio_input = "" #@param {type:"string"}
#@markdown You can paste the link to the video/audio from many sites, check the complete list [here](https://github.com/yt-dlp/yt-dlp/blob/master/supportedsites.md)

#@markdown Output path for audio files:
output_folder = "/content/Vocales" #@param {type:"string"}
#@markdown Select the model:
model = "BS-Roformer-Viperx-1297" #@param ["BS-Roformer-Viperx-1297", "BS-Roformer-Viperx-1296", "BS-Roformer-Viperx-1053", "Mel-Roformer-Viperx-1143", "BS-Roformer-De-Reverb", "Mel-Roformer-Crowd-Aufr33-Viperx", "Mel-Roformer-Denoise-Aufr33", "Mel-Roformer-Denoise-Aufr33-Aggr", "Mel-Roformer-Karaoke-Aufr33-Viperx", "MelBand Roformer | Vocals by Kimberley Jensen", "MelBand Roformer Kim | FT by unwa", "MelBand Roformer Kim | Inst V1 by Unwa", "MelBand Roformer Kim | Inst V1 (E) by Unwa", "MelBand Roformer Kim | Inst V2 by Unwa", "MelBand Roformer Kim | InstVoc Duality V1 by Unwa", "MelBand Roformer Kim | InstVoc Duality V2 by Unwa", "MelBand Roformer | De-Reverb by anvuew", "MelBand Roformer | De-Reverb Less Aggressive by anvuew", "MelBand Roformer | De-Reverb-Echo by Sucial", "MelBand Roformer | De-Reverb-Echo V2 by Sucial", "MelBand Roformer Kim | SYHFT by SYH99999", "MelBand Roformer Kim | SYHFT V2 by SYH99999", "MelBand Roformer Kim | SYHFT V2.5 by SYH99999", "MelBand Roformer Kim | SYHFT V3 by SYH99999", "MelBand Roformer Kim | Big SYHFT V1 by SYH99999", "MelBand Roformer Kim | Big Beta 4 FT by unwa", "MelBand Roformer Kim | Big Beta 5e FT by unwa", "BS Roformer | Chorus Male-Female by Sucial", "MelBand Roformer | Aspiration by Sucial", "MelBand Roformer | Aspiration Less Aggressive by Sucial", "MelBand Roformer | Bleed Suppressor V1 by unwa-97chris"]
#@markdown Select the output format:
output_format = "wav" #@param ["wav", "flac", "mp3", "ogg", "opus", "m4a", "aiff", "ac3"]
#@markdown Larger consumes more resources, but may give better results.
segment_size = 256 #@param {type:"slider", min:32, max:4000, step:32}
#@markdown Amount of overlap between prediction windows.
overlap = 8 #@param {type:"slider", min:2, max:10, step:1}
#@markdown Larger consumes more RAM but may process slightly faster.
batch_size = 1 #@param {type:"slider", min:1, max:16, step:1}
#@markdown Override model default segment size instead of using the model default value.
override_model_segment_size = False #@param {type:"boolean"}
#@markdown Flag to use PyTorch autocast for faster inference. Do not use for CPU inference.
use_autocast = True #@param {type:"boolean"}
extensions = (".wav", ".flac", ".mp3", ".ogg", ".opus", ".m4a", ".aiff", ".ac3")

uvr_cli(audio_input, output_folder, model, output_format, segment_size, overlap, batch_size, override_model_segment_size, use_autocast, extensions)

In [ ]:
#@title Download models

import os
import requests
from tqdm import tqdm

def download_model(url, model_name):
  """Downloads a model from a URL to a specified directory.

  Args:
    url: The URL of the model to download.
    model_name: The name of the model.
  """

  model_dir = os.path.join("models", model_name)
  os.makedirs(model_dir, exist_ok=True)
  zip_path = os.path.join(model_dir, "model.zip")

  try:
      response = requests.get(url, stream=True)
      response.raise_for_status() # Raise an exception for bad status codes

      total_size = int(response.headers.get('content-length', 0))
      block_size = 1024 # 1 KB
      progress_bar = tqdm(total=total_size, unit='iB', unit_scale=True)

      with open(zip_path, 'wb') as file:
          for data in response.iter_content(block_size):
              progress_bar.update(len(data))
              file.write(data)
      progress_bar.close()

      if total_size != 0 and progress_bar.n != total_size:
          print("ERROR, something went wrong")

      print(f"Model '{model_name}' downloaded successfully to {model_dir}")

  except requests.exceptions.RequestException as e:
      print(f"Error downloading model: {e}")
  except Exception as e:
      print(f"An unexpected error occurred: {e}")


# Example usage
model_url = "" #@param {type:"string"}
model_name = ""  #@param {type:"string"}
# Replace with your desired model name


download_model(model_url, model_name)

In [ ]:




#@title ## Inference
import os
main_dir = os.getcwd()
os.chdir(main_dir)
from lib.infer import infer_audio
from google.colab import files
from pydub import AudioSegment
import shutil

MODEL_NAME = "" #@param {type:"string"}
SOUND_PATH = "" #@param {type:"string"}
F0_CHANGE = 0 #@param {type:"integer"}
F0_METHOD = "fcpe" #@param ["crepe", "harvest", "mangio-crepe", "rmvpe", "rmvpe+", "fcpe", "fcpe_legacy", "hybrid[mangio-crepe+rmvpe]", "hybrid[mangio-crepe+fcpe]", "hybrid[rmvpe+fcpe]", "hybrid[mangio-crepe+rmvpe+fcpe]"] {allow-input:true}
CREPE_HOP_LENGTH = 120 #@param {type:"integer"}
INDEX_RATE = 0.75 #@param {type:"number"}
FILTER_RADIUS = 3 #@param {type:"integer"}
RMS_MIX_RATE = 0.25 #@param {type:"number"}
PROTECT = 0.33 #@param {type:"number"}
SPLIT_INFER = False #@param {type:"boolean"}
MIN_SILENCE = 500 #@param {type:"number"}
SILENCE_THRESHOLD = -50 #@param {type:"number"}
SEEK_STEP = 1 #@param {type:"slider", min:1, max:10, step:1}
KEEP_SILENCE = 200 #@param {type:"number"}
QUEFRENCY = 0 #@param {type:"number"}
TIMBRE = 1 #@param {type:"number"}
F0_AUTOTUNE = False #@param {type:"boolean"}
OUTPUT_FORMAT = "wav" #@param ["wav", "flac", "mp3"]



inferred_audio = infer_audio(
    MODEL_NAME,
    SOUND_PATH,
    F0_CHANGE,
    F0_METHOD,
    CREPE_HOP_LENGTH,
    INDEX_RATE,
    FILTER_RADIUS,
    RMS_MIX_RATE,
    PROTECT,
    SPLIT_INFER,
    MIN_SILENCE,
    SILENCE_THRESHOLD,
    SEEK_STEP,
    KEEP_SILENCE,
    QUEFRENCY,
    TIMBRE,
    F0_AUTOTUNE,
    OUTPUT_FORMAT
)
os.chdir(main_dir)

print(f"Showing {inferred_audio}.")
AudioSegment.from_file(inferred_audio)